In [1]:
import pandas as pd 
import numpy as np 
import os 
import cv2
import matplotlib.pyplot as plt
import ast


### save nodule masks 

In [2]:
test_csv = pd.read_csv("/fast_data_e2e_1/cxr/qxr_ln_data/LN_test/combined_test_csv_updated_internal_test_13-08-24.csv")

In [3]:
test_csv_nlst_jsrt  = test_csv[test_csv['src'] == 'nlst_jsrt'].reset_index(drop=True)

In [4]:
test_csv_nlst_jsrt.LNMS.value_counts()

LNMS
0    687
1    134
Name: count, dtype: int64

In [7]:
df_sample = test_csv_nlst_jsrt[test_csv_nlst_jsrt['bbox'].apply(lambda x: len(ast.literal_eval(x)) > 0)]
len(df_sample)

154

In [ ]:


# Assuming df is your DataFrame
df_sample = test_csv_nlst_jsrt[test_csv_nlst_jsrt['bbox'].apply(lambda x: len(ast.literal_eval(x)) > 0)].sample(5)
pixel_scale = 0.175

for index, row in df_sample.iterrows():
    # Fetch the image path
    image_path = row['png_path']
    
    # Load the image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    print(np.shape(image))
    
    # Extract bbox and convert to float
    bbox = ast.literal_eval(row['bbox'])[0]  # Assuming there's only one bbox per image
    
    # Extract the center coordinates and dimensions
    x_center, y_center, width, height = float(bbox[0]), float(bbox[1]), float(float(bbox[2]) / pixel_scale), float(float(bbox[3]) / pixel_scale)
    print(x_center, y_center, width, height)
    
    # Calculate top-left corner
    x_min = x_center - width / 2
    y_min = y_center - height / 2
    
    # Calculate bottom-right corner
    x_max = x_center + width / 2
    y_max = y_center + height / 2
    
    # Draw rectangle
    cv2.rectangle(image, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (255, 0, 0), 2)
    
    # Display the image
    plt.figure(figsize=(15, 15))
    plt.imshow(image)
    plt.title(f"Image with BBox: {image_path}")
    plt.axis('off')
    plt.show()


In [51]:

# Define the directory to save the masks
mask_save_dir = "/fast_data_e2e_1/cxr/qxr_ln_data/jsrt_nlst/masks"
os.makedirs(mask_save_dir, exist_ok=True)

# Assuming df is your DataFrame
df_sample = test_csv_nlst_jsrt[test_csv_nlst_jsrt['bbox'].apply(lambda x: len(ast.literal_eval(x)) > 0)]

pixel_scale = 0.175

for index, row in df_sample.iterrows():
    # Fetch the image path
    image_path = row['png_path']
    
    # Load the image to get the dimensions
    image = cv2.imread(image_path)
    height, width = image.shape[:2]
    
    # Create an empty binary mask
    mask = np.zeros((height, width), dtype=np.uint8)
    
    # Extract bbox and convert to float
    bbox = ast.literal_eval(row['bbox'])[0]  # Assuming there's only one bbox per image
    
    # Extract the center coordinates and dimensions
    x_center, y_center, bbox_width, bbox_height = float(bbox[0]), float(bbox[1]), float(float(bbox[2]) / pixel_scale), float(float(bbox[3]) / pixel_scale)
    
    # Calculate top-left corner
    x_min = int(x_center - bbox_width / 2)
    y_min = int(y_center - bbox_height / 2)
    
    # Calculate bottom-right corner
    x_max = int(x_center + bbox_width / 2)
    y_max = int(y_center + bbox_height / 2)
    
    # Ensure coordinates are within image boundaries
    x_min = max(0, x_min)
    y_min = max(0, y_min)
    x_max = min(width, x_max)
    y_max = min(height, y_max)
    
    # Draw the rectangle (filled) on the mask
    cv2.rectangle(mask, (x_min, y_min), (x_max, y_max), 255, thickness=-1)
    
    # Generate the mask file path
    mask_filename = os.path.splitext(os.path.basename(image_path))[0] + '_mask.png'
    mask_filepath = os.path.join(mask_save_dir, mask_filename)
    
    # Save the mask
    cv2.imwrite(mask_filepath, mask)
    
    print(f"Saved mask for {image_path} as {mask_filepath}")


Saved mask for /fast_data_e2e_1/cxr/qxr_ln_data/jsrt_nlst/data/JPCLN001.png as /fast_data_e2e_1/cxr/qxr_ln_data/jsrt_nlst/masks/JPCLN001_mask.png
Saved mask for /fast_data_e2e_1/cxr/qxr_ln_data/jsrt_nlst/data/JPCLN002.png as /fast_data_e2e_1/cxr/qxr_ln_data/jsrt_nlst/masks/JPCLN002_mask.png
Saved mask for /fast_data_e2e_1/cxr/qxr_ln_data/jsrt_nlst/data/JPCLN003.png as /fast_data_e2e_1/cxr/qxr_ln_data/jsrt_nlst/masks/JPCLN003_mask.png
Saved mask for /fast_data_e2e_1/cxr/qxr_ln_data/jsrt_nlst/data/JPCLN004.png as /fast_data_e2e_1/cxr/qxr_ln_data/jsrt_nlst/masks/JPCLN004_mask.png
Saved mask for /fast_data_e2e_1/cxr/qxr_ln_data/jsrt_nlst/data/JPCLN005.png as /fast_data_e2e_1/cxr/qxr_ln_data/jsrt_nlst/masks/JPCLN005_mask.png
Saved mask for /fast_data_e2e_1/cxr/qxr_ln_data/jsrt_nlst/data/JPCLN006.png as /fast_data_e2e_1/cxr/qxr_ln_data/jsrt_nlst/masks/JPCLN006_mask.png
Saved mask for /fast_data_e2e_1/cxr/qxr_ln_data/jsrt_nlst/data/JPCLN007.png as /fast_data_e2e_1/cxr/qxr_ln_data/jsrt_nlst/ma

### save lung_masks